In [5]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold

In [7]:
# stroke, 需要附上TeLU的公式
class TeLU(nn.Module):
    
    def __int__(self,alpha = 0.15):
        super(TeLU,self).__init__()
        self.alpha = alpha
        
    def forward(self,x):
        return torch.where(x >= 0, x, self.alpha * (torch.exp(x) - 1))

class FFNN(nn.Module):
    
    def __init__(self,input_size):
        super(FFNN,self).__init__()
        self.fc1 = nn.Linear(input_size,32)
        self.telu1 = TeLU(alpha = 0.15)
        self.fc2 = nn.Linear(32,64)
        self.telu2 = TeLU(alpha = 0.1)
        self.fc3 = nn.Linear(64,2)
        
    def forward(self,x):
        x = self.fc1(x)
        x = self.telu1(x)
        x = self.fc2(x)
        x = self.telu2(x)
        x = self.fc3(x)
        return x
    
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
    def forward(self, inputs, targets):
        BCE_loss = nn.CrossEntropyLoss(reduction='none')(inputs, targets)
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss
        return F_loss.mean()

In [8]:
data = pd.read_csv('preparations/stroke_output.csv')

data.head()
# 标准化，还需要处理不平衡集的问题
def train_test_split(X,y, splits = 10, epochs = 500, batch_size = 512, lr = 0.001):
    
    k_fold_10 = StratifiedKFold(X,y,n_splits= 10,shuffle=True,random_state=2025, patience=20, model_save_path = 'best_model_stroke.pth')
    result = []
    for fold, (train_idx, val_idx) in enumerate(k_fold.split(X, y)):
        print(f'\nFold {fold+1}/{splits}')
        
        # Split data
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        
        # Standardize
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_val = scaler.transform(X_val)
        
        
        

,age,hypertension,heart_disease,ever_married,work_type,avg_glucose_level,bmi,smoking_status,stroke
0,67.0,0,1,1,3.0,228.69,36.600000,1,1
1,61.0,0,0,1,2.0,202.21,28.893237,0,1
2,80.0,0,1,1,3.0,105.92,32.500000,0,1
3,49.0,0,0,1,3.0,171.23,34.400000,2,1
4,79.0,1,0,1,2.0,174.12,24.000000,0,1
